In [ ]:
!pip install datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [ ]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer

model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/INTERIIT/bert-trained2',device_map="cuda",num_labels=1)

tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/INTERIIT/bert-trained2')

In [ ]:
from sklearn.preprocessing import StandardScaler

# Using the parameters obtained from the previous code
scaler_params = {
    'copy': True,  # Example parameter from the obtained scaler_params
    'with_mean': True,  # Example parameter from the obtained scaler_params
    'with_std': True,
}

# Create a new scaler and set its parameters
scaler = StandardScaler()
scaler.set_params(**scaler_params)
scaler_means = np.loadtxt('/content/drive/MyDrive/INTERIIT/scaler_means.txt')
scaler_std_devs = np.loadtxt('/content/drive/MyDrive/INTERIIT/scaler_means.txt')

# Set up a new scaler with the loaded mean and standard deviation values
scaler = StandardScaler()
scaler.mean_ = scaler_means
scaler.scale_ = scaler_std_devs

# Image Caption

In [ ]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16
)  # doctest: +IGNORE_RESULT

In [ ]:
img_url = 'https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=small'
raw_image = Image.open(requests.get(img_url, stream=True).raw)

prompt = "Question: What is the picture showing? Answer:"
inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to(device="cuda", dtype=torch.float16)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

# Multiple Dataset prediction

In [ ]:
df_result1=pd.read_excel('/content/drive/MyDrive/INTERIIT/behaviour_simulation_test_company.xlsx')
df_result1 = df_result1.reset_index(drop=True)
df_result1.dropna(inplace=True)

In [ ]:
df_result1['text'] = (
    "Input: \n"
    "Tweet was posted by " + df_result1['username'] + " on Twitter with the content '" +
    df_result1['content'] + "' on " + df_result1['date'] + ". The inferred company based on the tweet was " +
    df_result1['inferred company'] + " company.  Predict the amount of likes that the tweet got anywhere from \"0\" to \"2000\", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
)
df_result1

,id,date,content,username,media,inferred company,Predicted Results,text
0,1,2019-01-08 18:03:10,the Heathrow drone right now <hyperlink>,bbcthree,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc,583,Input: \nTweet was posted by bbcthree on Twitt...
1,2,2018-01-29 10:51:17,A deal at £60m?\n\nThe latest on Pierre-Emeric...,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc,575,Input: \nTweet was posted by BBCSport on Twitt...
2,3,2019-09-05 07:25:14,"""Dealing with Boris Johnson is like someone de...",BBCPolitics,[Video(thumbnailUrl='https://pbs.twimg.com/med...,bbc,767,Input: \nTweet was posted by BBCPolitics on Tw...
3,4,2018-03-28 18:15:00,Doesn't unlimited food &amp; drinks all night ...,BrdgstoneArena,[Photo(previewUrl='https://pbs.twimg.com/media...,bridgestone,838,Input: \nTweet was posted by BrdgstoneArena on...
4,5,2019-09-11 21:15:44,Apple Watch Series 5.\n\nYou’ve never seen a w...,BestBuy,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,best buy,605,Input: \nTweet was posted by BestBuy on Twitte...
...,...,...,...,...,...,...,...,...
9995,9996,2020-01-25 01:45:07,"""So what do all these defenses mean? What they...",HouseIntelDems,[Video(thumbnailUrl='https://pbs.twimg.com/med...,house,362,Input: \nTweet was posted by HouseIntelDems on...
9996,9997,2019-06-26 12:51:27,"In this week’s #MacroMemo, we explore last wee...",rbcgamnews,[Photo(previewUrl='https://pbs.twimg.com/media...,rbc,579,Input: \nTweet was posted by rbcgamnews on Twi...
9997,9998,2019-05-06 10:37:03,When it's bank holiday Monday and someone ment...,bbceastenders,[Gif(thumbnailUrl='https://pbs.twimg.com/tweet...,bbc,528,Input: \nTweet was posted by bbceastenders on ...
9998,9999,2019-05-19 12:00:51,England boss Gareth Southgate says he is conce...,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc,1240,Input: \nTweet was posted by BBCSport on Twitt...


In [ ]:
columns_to_drop = ['id', 'username', 'inferred company','media','content','date']
df_result1.drop(columns=columns_to_drop, inplace=True)
df_result1

,Predicted Results,text
0,583,Input: \nTweet was posted by bbcthree on Twitt...
1,575,Input: \nTweet was posted by BBCSport on Twitt...
2,767,Input: \nTweet was posted by BBCPolitics on Tw...
3,838,Input: \nTweet was posted by BrdgstoneArena on...
4,605,Input: \nTweet was posted by BestBuy on Twitte...
...,...,...
9995,362,Input: \nTweet was posted by HouseIntelDems on...
9996,579,Input: \nTweet was posted by rbcgamnews on Twi...
9997,528,Input: \nTweet was posted by bbceastenders on ...
9998,1240,Input: \nTweet was posted by BBCSport on Twitt...


In [ ]:
result_dataset= Dataset.from_pandas(df_result1)

In [ ]:
import math
import torch


nb_batches = math.ceil(len(result_dataset)/64)
y_preds = []

for i in range(nb_batches):
    input_texts = result_dataset[i * 64: (i+1) * 64]["text"]
    # input_labels = result_dataset[i * 200: (i+1) * BATCH_SIZE]["scaled_likes"]
    encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=128, add_special_tokens=True)
    input_ids = torch.tensor(encoded['input_ids']).to('cuda:0') # Convert input_ids to a tensor
    attention_mask = torch.tensor(encoded['attention_mask']).to('cuda:0') # Assuming the tokenizer returns attention_mask

    batch_y_preds = model(input_ids, attention_mask=attention_mask).logits

    # Append predictions for the current batch to y_preds list
    y_preds += batch_y_preds.detach().cpu().tolist()
    # encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=256,add_special_tokens=True)
    # y_preds = model(**encoded).logits

# pd.set_option('display.max_rows', 500)
# test_df["likes"] = test_df["scaled_likes"] * (max_likes - min_likes) + min_likes
# df_out = pd.DataFrame([test_dataset["text"], test_df["likes"], y_preds_rescaled], ["Text", "likes", "Prediction"])

In [ ]:
y_preds_np = np.array(y_preds).reshape(-1, 1)

In [ ]:
y_preds_original_scale = scaler.inverse_transform(y_preds_np)

In [ ]:
y_preds_int = np.round(y_preds_original_scale).astype(int)

In [ ]:
y_preds_int

array([[751],
       [758],
       [816],
       ...,
       [775],
       [775],
       [760]])

In [ ]:
with open("/content/drive/MyDrive/INTERIIT/results.txt", 'w') as output:
    for val in y_preds_int:
        output.write(str(val) + '\n')

# Prediction on a single text

In [ ]:
input_texts = "Chapter 246: The Decisive Battle in the Uninhabited, Demon-Infested Shinjuku ⑱ of Jujutsu Kaisen is out!  Be mindful online of official only readers! Link to the Chapter in our Bio. Predict the amount of likes that the tweet got anywhere from \"0\" to \"2000\", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes."

In [ ]:
import torch

def predict_single_text(text, model, tokenizer):
    # Tokenize the input text
    encoded = tokenizer(text, truncation=True, padding="max_length", max_length=128, add_special_tokens=True)
    input_ids = torch.tensor(encoded['input_ids']).unsqueeze(0).to('cuda:0')  # Add batch dimension and move to GPU
    attention_mask = torch.tensor(encoded['attention_mask']).unsqueeze(0).to('cuda:0')  # Add batch dimension and move to GPU

    # Make the prediction for the single text
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask).logits

    return output.detach().cpu().tolist()

In [ ]:
y = predict_single_text(input_texts,model,tokenizer)

In [ ]:
y_original = scaler.inverse_transform(y)

In [ ]:
y_int = np.round(y_original).astype(int)

In [ ]:
print(y_int[0][0])

776
